In [ ]:
# Array shape conventions:
# d = vector dimension (6 or 7, usually)
# T = number of time points (~10000, usually)
# N = number of spins simulated
# 1. Vector time series: (d, T)
# 2. Ensemble: (d, T, N)

In [ ]:
module_path = "/home/rtat/spin-simulation/"
if findfirst(x -> x == module_path, LOAD_PATH) == nothing
    push!(LOAD_PATH, module_path)
end
using Revise

In [ ]:
using Utils
using Solve
using Analysis
using Plots
using DSP

In [ ]:
# Holds critical dressing parameters. B0 and B1 are in [Gauss], w is angular frequency
# B(t) = B0 \hat{z} + B1 cos(wt) \hat{x}
crit_params 

In [ ]:
sol = run_simulations(1, 2; nsave=10000); # Simulate 2 neutron-3He pairs for 1 second.
# By default, B0, B1, and w are set to critical dressing values.
# By default, a small amount of white noise is applied along the B1 direction.

In [ ]:
sol.t

In [ ]:
# The first 3 rows are the neutron spin Bloch vector components
# The next 3 rows correspond to the 3He Bloch vector.
sol.u

In [ ]:
# Make a directory to save things in
save_dir = "data/results"
if !isdir(save_dir)
    mkdir(save_dir)
end

In [ ]:
sim_time = 1 # Simulation time in seconds
n_runs = 50 # Number of runs to simulate. Noise is randomized each time.
nsave = 10001
initial_phases = (pi/4, -pi/4) # Angle in the x-y plane to start the spins at. Relative to x-axis.

In [ ]:
#= 
The noisy part of the B field is simulated as Gaussian random points 
placed at fixed intervals in time, which are then interpolated
to form a continuous function. 

Bnoise = sigma of the Gaussian
noiserate = # of points per second.

In this code, the noisy B field is always taken to be in the x direction.
=#
Bnoise=crit_params["B1"]*2e-3
noiserate = 20000

#=
The code uses filters to modify the power spectrum of the noise
=#
filterorder = 7
filterripple = 0.1
filterdB = 60
filtertype = Elliptic(filterorder, filterripple, filterdB)
uppercutoff = 0 # No upper cutoff

In [ ]:
for lowercutoff=[0, 30, 60, 200, 500, 2000] # Various highpass filter cutoffs
    # Simulate a run with no noise
    no_noise_sol = run_simulations(sim_time, 1;
        initial_phases=initial_phases,
        Bnoise=0,
        nsave=nsave,
        )

    # Simulate several runs with noise
    sol = run_simulations(sim_time, n_runs;
        Bnoise=Bnoise,
        noiserate=noiserate,
        filtertype=filtertype,
        lowercutoff=lowercutoff,
        uppercutoff=uppercutoff,
        initial_phases=initial_phases,
        nsave=nsave)

    # Save some metadata for future reference
    metadata = Dict{String, Any}("time"=>sim_time,
        "Bnoise"=>Bnoise,
        "lowercutoff"=>lowercutoff,
        "uppercutoff"=>uppercutoff, 
        "initial_phases"=>initial_phases,
        "filterorder"=>filterorder,
        "filterripple"=>filterripple,
        "filterdB"=>filterdB,
        "noiserate"=>noiserate)

    # Convenience function saves these objects to a directory inside save_dir
    # Returns the name of the newly created directory.
    most_recent_save = save_data(no_noise_sol, sol, metadata, save_dir)
end 

In [ ]:
most_recent_save

In [ ]:
# Convenience function for loading data
no_noise_sol, sol, metadata = load_data(most_recent_save);